# one cell Confusion matrix tool: 

In [ ]:
'''
This is a compact from of the Confusion matrix tool notebook, it runs in one cell or can be run on Raijin. It is designed to be pointed at a file with all of the DLCD tif and re-run with different year inputs by changing one variable:  the 'year_in' which should be the first year of the 2 year span of each DLCD map. 

this variable only needs to be changed once, it goes into the filenames called and created

This notebook is designed to detect change between different itterations of the DLCD, 
using as little compute power as posible.

as a result, this notebook dosn't have any images, however they can be easily generated from the xarrays found within. To run this notebook you will need acess to the DLCD itterations in the form of geotifs. The out put is a confusion matrix in CSV file format. 

In the output CSV file the landcover types for the old itteration are along the X axis, (across the top of the file in the first row) the landcover types for the new itteration run down the Y axis (in the first collum of the file) ie:

    old old old old old old old ...
new 100   0   0   2  31   0  12 ...
new   0 200   0   0   1   1   0 ...
new   1   0 150   1   4   0   0 ...
...

'''


#Enter year

year_in = '2010'

#load requiered modules 
import numpy as np
import rasterio
#from matplotlib import pyplot as plt #don't need unless want to plot
import xarray as xr
import pandas as pd

#load in geotifs using rasterio, trun into an xarray :
#enter filename as a string, include subfolders!!
#previous_version DLCD found in old_DLCD, new in new_DLCD.

#use this command to print a list of all file in respective folder : 

def load_gfif(file_name, x):
    #file_string = 'old_DLCD/'+file_name+'.tif'

    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    #year_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_array) #return np array into program 
    
second_year = int(year_in)
second_year = second_year + 1
second_year = str(second_year)

old_dlcd_string='old_DLCD/DLCDv2_'+year_in+'_'+second_year+'1.tif'
new_dlcd_band= int(year_in[1:4])

old_version = load_gfif(old_dlcd_string,1)#for old dlcd files all only have ONE band (are single year)

new_version = load_gfif('new_DLCD/DLCDv2_2016_new.tif',new_dlcd_band)#for new DLCD band number corrosponds to year in following table

# define function to return the number of pixles which are class A in old and class B in new: 

def count_overlap(old_np,new_np,A,B):
    raw_np = np.ones_like(old_np)
    raw_xr = xr.DataArray(raw_np)
    count_old = raw_xr.where(old_np==A)
    count_new = count_old.where(new_np==B)
    count_class = count_new.fillna(0) #replaces all the nan with 0
    howmuch_name = np.count_nonzero(count_class) #counts the number of non 0 in the xaray


    return(howmuch_name) #returns an int which is the number of pixles that are A in old and B in new

class_values = [
1,
3,
4,
5,
6,
7,
8,
9,
10,
11,
14,
15,
16,
18,
19,
22,    
24,
25,
31,
32,
33,
34,
35
]

matrix_d = dict()

for i in class_values:
    i_lable = int(i)
    d = dict()
    for x in class_values:
        x_lable =int(x)
        change = count_overlap(old_version,new_version,i,x)
        d[x_lable] = change
        #print(i,x,change)
    matrix_d[i_lable] = d
    #print('added')

matrix_dataframe = pd.DataFrame(matrix_d)


year_in
file_name = 'Matrixes/confustion_matrix_for_'+year_in+'.csv'

matrix_dataframe.to_csv(file_name)

